In [1]:
import pandas as pd
import json
import re

In [ ]:
FIXTURE_PATH = "./data/fixtures-2023-2024.json"


with open(FIXTURE_PATH, "r") as file:
    fixtures = json.load(file)

In [3]:
len(fixtures["result"])

380

In [4]:
list(fixtures["result"][0].keys())

['event_key',
 'event_date',
 'event_time',
 'event_home_team',
 'home_team_key',
 'event_away_team',
 'away_team_key',
 'event_halftime_result',
 'event_final_result',
 'event_ft_result',
 'event_penalty_result',
 'event_status',
 'country_name',
 'league_name',
 'league_key',
 'league_round',
 'league_season',
 'event_live',
 'event_stadium',
 'event_referee',
 'home_team_logo',
 'away_team_logo',
 'event_country_key',
 'league_logo',
 'country_logo',
 'event_home_formation',
 'event_away_formation',
 'fk_stage_key',
 'stage_name',
 'league_group',
 'goalscorers',
 'substitutes',
 'cards',
 'vars',
 'lineups',
 'statistics',
 'player_stats']

# Players

In [5]:
player_df = pd.DataFrame(
    columns=[
        "PlayerID",
        "PlayerName",
        "height",
        "PlayerBirthdate",
        "PlayerPosition",
        "TeamID",
        "AwardID",
        "StatsID",
    ]
)

players = {}
for fixture in fixtures["result"]:
    player_stats = fixture.get("player_stats")

    home_team_key = fixture.get("home_team_key")
    home_players = player_stats.get("home")

    away_team_key = fixture.get("away_team_key")
    away_players = player_stats.get("away")

    for player in home_players:
        player_name = player.get("player_name")
        player_key = player.get("player_key")
        player_position = player.get("player_position")
        team_id = home_team_key

        if not player_key in players:
            players[player_key] = {
                "PlayerID": player_key,
                "PlayerName": player_name,
                "height": 0.0,
                "PlayerBirthdate": "",
                "PlayerPosition": player_position,
                "TeamID": str(team_id),
                "AwardID": "",
                "StatsID": "",
            }

    for player in away_players:
        player_name = player.get("player_name")
        player_key = player.get("player_key")
        player_position = player.get("player_position")
        team_id = away_team_key

        if not player_key in players:
            players[player_key] = {
                "PlayerID": player_key,
                "PlayerName": player_name,
                "height": 0.0,
                "PlayerBirthdate": "",
                "PlayerPosition": player_position,
                "TeamID": str(team_id),
                "AwardID": "",
                "StatsID": "",
            }

In [6]:
len(players.keys())

732

In [7]:
temp_players = []
for key, value in players.items():
    temp_players.append(value)

player_df = pd.DataFrame(temp_players)

In [8]:
player_df.head()

,PlayerID,PlayerName,height,PlayerBirthdate,PlayerPosition,TeamID,AwardID,StatsID
0,3446405013,David Raya,0.0,,Goalkeepers,141,,
1,1058876140,Benjamin White,0.0,,Defenders,141,,
2,1690625216,William Saliba,0.0,,Defenders,141,,
3,1215841429,Gabriel Magalhães,0.0,,Defenders,141,,
4,2999851979,Takehiro Tomiyasu,0.0,,Defenders,141,,


In [57]:
player_df.to_excel("./crawled_dataset/player_dataset.xlsx", index=False)

# Match

In [64]:
matches = {}
referees = {}
performancestats = {}


for fixture in fixtures["result"]:
    event_key = fixture.get("event_key")
    home_team_key = fixture.get("home_team_key")
    away_team_key = fixture.get("away_team_key")
    event_final_result = fixture.get("event_final_result")
    event_date = fixture.get("event_date")
    event_stadium = fixture.get("event_stadium")

    if event_key not in matches:
        matches[event_key] = {
            "MatchID": event_key,
            "MatchHomeTeam": home_team_key,
            "MatchAwayTeam": away_team_key,
            "Score": event_final_result,
            "Date": event_date,
            "StadiumID": event_stadium,
        }

In [65]:
match_df = pd.DataFrame([value for key, value in matches.items()])

In [71]:
match_df.shape

(380, 6)

In [ ]:
match_df.to_excel("./data/match_dataset.xlsx", index=False)

# Referee

In [38]:
referees = {}

for fixture in fixtures["result"]:
    event_key = fixture.get("event_key")
    event_referee = fixture.get("event_referee")

    referees[event_key] = {
        "RefereeID": "",
        "RefereeName": event_referee,
        "RefereeNationality": "",
        "RBirthdate": "",
        "MatchID": event_key,
    }

In [39]:
referees_df = pd.DataFrame([value for key, value in referees.items()])

In [40]:
print(referees_df.shape)
referees_df.head()

(380, 5)


,RefereeID,RefereeName,RefereeNationality,RBirthdate,MatchID
0,,M. Oliver,,,1226163
1,,S. Hooper,,,1226164
2,,C. Pawson,,,1226165
3,,G. Scott,,,1226166
4,,A. Taylor,,,1226167


In [ ]:
referees_df.to_excel("./data/referee_dataset.xlsx", index=False)

# Stadium

In [74]:
stadiums = {}

for fixture in fixtures["result"]:
    event_stadium = fixture.get("event_stadium")
    country_name = fixture.get("country_name")
    event_home_team = fixture.get("event_home_team")
    stadium_region = ""
    
    match = re.search(r"\((.*?)\)", event_stadium)
    if match:
        stadium_region = f"{match.group(1)} {country_name}"
    else:
        stadium_region = country_name
        
    if event_stadium:
        stadiums[event_stadium] = {
            "StadiumID": "",
            "StadiumName": event_stadium,
            "StadiumRegion": stadium_region,
            "HomeTeam": event_home_team,
            "Capacity": "",
            "OpeningYear": "",
        }

In [75]:
stadiums_df = pd.DataFrame([value for key, value in stadiums.items()])

In [76]:
stadiums_df.shape

(21, 6)

In [77]:
stadiums_df.head()

,StadiumID,StadiumName,StadiumRegion,HomeTeam,Capacity,OpeningYear
0,,Emirates Stadium (London),London England,Arsenal,,
1,,"Gtech Community Stadium (Brentford, Middlesex)","Brentford, Middlesex England",Brentford,,
2,,"American Express Stadium (Falmer, East Sussex)","Falmer, East Sussex England",Brighton & Hove Albion,,
3,,Turf Moor (Burnley),Burnley England,Burnley,,
4,,Stamford Bridge (London),London England,Chelsea,,


In [78]:
stadiums_df.to_excel("./crawled_dataset/stadium_dataset.xlsx", index=False)

# Performance stats

In [ ]:
from uuid import uuid4

performancestats = {}

for fixture in fixtures["result"]:
    event_key = fixture.get("event_key")
    league_season = fixture.get("league_season")
    player_stats = fixture.get("player_stats")
    home_stats = player_stats.get("home")
    home_stats = player_stats.get("home")
    away_stats = player_stats.get("away")

    for home_stat in home_stats:
        player_key = home_stat.get("player_key")
        player_goals = home_stat.get("player_goals")
        player_assists = home_stat.get("player_assists")
        player_yellow_cards = home_stat.get("player_yellow_cards")
        player_red_cards = home_stat.get("player_red_cards")
        player_fouls_commited = home_stat.get("player_fouls_commited")
        player_passes_acc = home_stat.get("player_passes_acc")

        if f"{player_key}_{event_key}" not in performancestats:
            performancestats[f"{player_key}_{event_key}"] = {
                "StatsID": uuid4().hex,
                "MatchID": event_key,
                "PlayerID": player_key,
                "GoalsScored": player_goals,
                "Assists": player_assists,
                "YellowCards": player_yellow_cards,
                "RedCards": player_red_cards,
                "Fouls": player_fouls_commited,
                "Pass Accuracy": player_passes_acc,
                "Season": league_season,
            }

    for away_stat in away_stats:
        player_key = away_stat.get("player_key")
        player_goals = away_stat.get("player_goals")
        player_assists = away_stat.get("player_assists")
        player_yellow_cards = away_stat.get("player_yellow_cards")
        player_red_cards = away_stat.get("player_red_cards")
        player_fouls_commited = away_stat.get("player_fouls_commited")
        player_passes_acc = away_stat.get("player_passes_acc")

        if f"{player_key}_{event_key}" not in performancestats:
            performancestats[f"{player_key}_{event_key}"] = {
                "StatsID": uuid4().hex,
                "MatchID": event_key,
                "PlayerID": player_key,
                "GoalsScored": player_goals,
                "Assists": player_assists,
                "YellowCards": player_yellow_cards,
                "RedCards": player_red_cards,
                "Fouls": player_fouls_commited,
                "Pass Accuracy": player_passes_acc,
                "Season": league_season,
            }

In [13]:
performancestats_df = pd.DataFrame([value for key, value in performancestats.items()])

In [14]:
performancestats_df.head()

,StatsID,MatchID,PlayerID,GoalsScored,Assists,YellowCards,RedCards,Fouls,Pass Accuracy,Season
0,99182aa688a042469f62b6251b3902c7,1226163,3446405013,0,0,0,0,0,13,2023/2024
1,8de45c1bd62b40abb1c8eed974042dd1,1226163,1058876140,0,0,0,0,0,62,2023/2024
2,f256bb96af864992aeaeba6e9944983c,1226163,1690625216,0,0,0,0,0,88,2023/2024
3,887df814e05440d89fff4c9a3aef25de,1226163,1215841429,0,0,0,0,0,39,2023/2024
4,174afbf350e44eacbc36ca6cad230dd8,1226163,2999851979,1,0,0,0,0,43,2023/2024


In [15]:
performancestats_df.shape

(15100, 10)

In [ ]:
performancestats_df.to_excel("./data/performancestats_dataset.xlsx", index=False)

# Coach

In [22]:
coaches = {}

for fixture in fixtures["result"]:
    event_key = fixture.get("event_key")

    home_team_key = fixture.get("home_team_key")
    event_home_team = fixture.get("event_home_team")
    away_team_key = fixture.get("away_team_key")
    event_away_team = fixture.get("event_away_team")

    lineups = fixture.get("lineups")
    home_lineups = lineups.get("home_team")
    away_lineups = lineups.get("away_team")

    event_home_formation = fixture.get("event_home_formation")
    event_away_formation = fixture.get("event_away_formation")

    home_coaches = home_lineups.get("coaches")
    home_coach = home_coaches[0].get("coache")

    away_coaches = away_lineups.get("coaches")
    away_coach = away_coaches[0].get("coache")

    if home_coach not in coaches:
        coaches[home_coach] = {
            "CoachID": "",
            "CoachName": home_coach,
            "CoachNationality": "",
            "CPreferredFormation": event_home_formation,
            "CPreviousTeams": event_home_team,
            "TeamID": home_team_key,
        }

    if away_coach not in coaches:
        coaches[away_coach] = {
            "CoachID": "",
            "CoachName": away_coach,
            "CoachNationality": "",
            "CPreferredFormation": event_away_formation,
            "CPreviousTeams": event_away_team,
            "TeamID": away_team_key,
        }

In [23]:
coaches_df = pd.DataFrame([value for key, value in coaches.items()])

In [24]:
coaches_df.shape

(23, 6)

In [25]:
coaches_df.head()

,CoachID,CoachName,CoachNationality,CPreferredFormation,CPreviousTeams,TeamID
0,,Mikel Arteta,,4-3-3,Arsenal,141
1,,S. Dyche,,4-4-1-1,Everton,3073
2,,T. Frank,,4-3-3,Brentford,3086
3,,E. Howe,,4-3-3,Newcastle United,3100
4,,R. De Zerbi,,4-2-3-1,Brighton & Hove Albion,3079


In [ ]:
coaches_df.to_excel("./data/coach_dataset.xlsx", index=False)

# Team

In [32]:
teams = {}

for fixture in fixtures["result"]:
    event_key = fixture.get("event_key")
    league_name = fixture.get("league_name")
    league_key = fixture.get("league_key")
    event_stadium = fixture.get("event_stadium")

    home_team_key = fixture.get("home_team_key")
    event_home_team = fixture.get("event_home_team")
    away_team_key = fixture.get("away_team_key")
    event_away_team = fixture.get("event_away_team")

    lineups = fixture.get("lineups")
    home_lineups = lineups.get("home_team")
    away_lineups = lineups.get("away_team")

    event_home_formation = fixture.get("event_home_formation")
    event_away_formation = fixture.get("event_away_formation")

    if event_home_team not in teams:
        teams[home_team_key] = {
            "TeamID": home_team_key,
            "TeamName": event_home_team,
            "TeamFormation": event_home_formation,
            "TeamLeague": league_name,
            "StadiumID": event_stadium,
            "TrophyID": "",
            "LeagueID": league_key,
            "TournamentID": 6,
            "TeamRanking": "",
            "TeamSeason": league_season,
        }

    if away_team_key not in teams:
        teams[away_team_key] = {
            "TeamID": away_team_key,
            "TeamName": event_away_team,
            "TeamFormation": event_away_formation,
            "TeamLeague": league_name,
            "StadiumID": event_stadium,
            "TrophyID": "",
            "LeagueID": league_key,
            "TournamentID": 6,
            "TeamRanking": "",
            "TeamSeason": league_season,
        }

In [33]:
teams_df = pd.DataFrame([value for key, value in teams.items()])

In [34]:
teams_df.shape

(20, 10)

In [35]:
teams_df.head()

,TeamID,TeamName,TeamFormation,TeamLeague,StadiumID,TrophyID,LeagueID,TournamentID,TeamRanking,TeamSeason
0,141,Arsenal,4-3-3,Premier League,Emirates Stadium (London),,152,6,,2023/2024
1,3073,Everton,4-4-1-1,Premier League,Goodison Park (Liverpool),,152,6,,2023/2024
2,3086,Brentford,5-3-2,Premier League,"Gtech Community Stadium (Brentford, Middlesex)",,152,6,,2023/2024
3,3100,Newcastle United,4-3-3,Premier League,St. James' Park (Newcastle upon Tyne),,152,6,,2023/2024
4,3079,Brighton & Hove Albion,4-2-3-1,Premier League,The American Express Community Stadium (Falmer...,,152,6,,2023/2024


In [ ]:
teams_df.to_excel("./data/team_dataset.xlsx", index=False)